In [1]:
import glob
import re
import spacy
import unicodedata
import csv

import pandas as pd

from collections import defaultdict
from minicons.utils import find_pattern
from spacy.lang.en import English
from tqdm import tqdm

import utils
import config
import minicons.utils as mu

from nltk.tokenize import sent_tokenize

In [5]:
nlp = English()
tokenizer = nlp.tokenizer

def tokenize(string):
    return [t.text for t in tokenizer(string)]

In [3]:
# load sents and postags
sent_dir = "/home/km55359/rawdata/babylm_data/babylm_100M/sents/"
# sent_dir = "/Users/kanishka/rawdata/babylm-sents-and-postags/"
sents = utils.read_file(f"{sent_dir}/babylm_sents.txt")
postags = utils.read_file(f"{sent_dir}/postags.txt")

# sent_tokens = [tokenize(sent) for sent in sents]

# # write sent tokens to file
# with open(f"{sent_dir}/sent_tokens.txt", "w") as f:
#     for sent in sent_tokens:
#         f.write(" ".join(sent) + "\n")

sent_tokens = utils.read_file(f"{sent_dir}/sent_tokens.txt")


In [4]:
PERMISSIVE = r'\b(a|an|another)\b .{0,100} \b(two|three|four|five|six|seven|eight|nine|ten|eleven|twelve|thirteen|fourteen|fifteen|sixteen|seventeen|eighteen|nineteen|twenty|thirty|forty|fifty|sixty|seventy|eighty|ninety|hundred|thousand|million|billion|several|few|couple|dozen|tens|dozens|hundreds|thousands|millions|billions|[0-9]+)\b.{0,100}\b\w+s\b'

CURRENT = r'\bDT\s(((HYPH|,)\s))?((((RB|CC)\s)+)?((JJ|JJR|JJS|VBN|((NN CC NN |NN HYPH )+(JJ|JJR|JJS|VBN)))((\s(HYPH|,))?)\s))+(((RB)\s)+)?(((HYPH|,)\s))?((UH)\s)?(((NN|CC)\s)+)?((CD)(\s(TO|CC|(HYPH|,))(\s(HYPH|,))?)?\s)+(((HYPH|,)\s))?(JJR\s)?((NNS|NNPS|(NN\sNNS)|((NN|NNS) IN NNS)))+'

CURRENT_DT = r'\bDT\s(((HYPH|,)\s))?((((RB|CC|IN)\s)+)?((JJ|JJR|JJS|VBN|((NN CC NN |NN HYPH )+(JJ|JJR|JJS|VBN)))((\s(HYPH|,))?)\s))+(((RB)\s)+)?(((HYPH|,)\s))?((UH)\s)?(((NN|CC)\s)+)?((CD)(\s(TO|CC|(HYPH|,))(\s(HYPH|,))?)?\s)+(((HYPH|,)\s))?(JJR\s)?(DT\s)?((NNS|NNPS|(NN\sNNS)|((NN|NNS) IN NNS)))+'

CURRENT_DEP_PARSE = r'\bDT\s(((HYPH|,)\s))?((((RB|CC|IN)\s)+)?((JJ|NN|JJR|JJS|VBN|((NN CC NN |NN HYPH )+(JJ|JJR|JJS|VBN)))((\s(HYPH|,))?)\s))+(((RB)\s)+)?(((HYPH|,)\s))?((UH)\s)?(((NN|CC)\s)+)?((CD)(\s(TO|CC|(HYPH|,))(\s(HYPH|,))?)?\s)+(((HYPH|,)\s))?(JJR\s)?((NNS|NNPS|(NN\sNNS)|((NN|NNS) IN NNS)))+'

In [83]:
non_numerals = ['few', 'dozen', 'couple', 'several', 'many', 'more']

CURRENT_FEW = r'\bARTICLE\s(((HYPH|,)\s))?((((RB|CC|IN)\s)+)?((JJ|JJR|JJS|VBN|RECORD|((NN CC NN |NN HYPH )+(JJ|JJR|JJS|VBN|RECORD)))((\s(HYPH|,))?)\s))+(((RB)\s)+)?(((HYPH|,)\s))?((UH)\s)?(((NN|CC)\s)+)?((CD|FEW)(\s(TO|CC|(HYPH|,))(\s(HYPH|,))?)?\s)+(((HYPH|,)\s))?((JJR|JJ|VBN)\s)?(ARTICLE\s)?((NNS|NNPS|(NN\sNNS)|((NN|NNS) IN NNS)))+'

def detect_aann_basic(sent_toks, pos):
    p_replaced = []
    for st, pt in zip(sent_toks, pos.split(" ")):
        if st.lower() not in non_numerals + ["a", "an", "another", "an", "-a", "-an"] + ['record']:
            p_replaced.append(pt)
        else:
            if st.lower() in ["a", "another", "an", "-a", "-an"]:
                p_replaced.append("ARTICLE")
            elif st.lower() in non_numerals:
                p_replaced.append("FEW")
            elif st.lower() in ["record"]:
                p_replaced.append("RECORD")
            else:
                p_replaced.append(pt)
        # if st.lower() in non_numerals:
        #     p_replaced.append("FEW")
        # else:
        #     p_replaced.append(pt)
        # if st.lower() in ["a", "another", "an", "-a", "-an"]:
        #     p_replaced.append("ARTICLE")
        # else:
        #     p_replaced.append(pt)
    postag_seq = " ".join(p_replaced)
    print(postag_seq)
    searched = re.search(CURRENT_FEW, postag_seq)
    result = (False, "")
    if searched:
        span = searched.span()
        construction_pattern = postag_seq[span[0] : span[1]]
        construction_pattern_span = find_pattern(
            construction_pattern.split(), postag_seq.split()
        )
        if sent_toks != []:
            try:
                sent_span = sent_toks[construction_pattern_span[0] : construction_pattern_span[1]]
                result = (True, " ".join(sent_span))
            except:
                pass
    return result

In [68]:
# few_pos = defaultdict(int)

# for i, (s, sts, p) in enumerate(tqdm(zip(sents, sent_tokens, postags))):
#     # align s to p
    
# permissed_sampled_annotated = utils.read_csv_dict("../data/permissed_sampled_annotated_old1k.csv")
permissed_sampled_annotated = utils.read_csv_dict("../data/permissed_sampled_annotated_test1_1k_predictions.csv")

# with open("../data/permissed_sampled_annotated_old1k_new_regex.csv", "w") as f:
#     writer = csv.writer(f)
#     writer.writerow(["sentence","pos","span","length","aann","og_detected","current_detected","dep_parse_detected","current_with_dt_detected", "final_detected"])
#     for i, entry in enumerate(permissed_sampled_annotated):
#         sent_tok = tokenize(entry['sentence'])
#         detected, span = detect_aann_basic(sent_tok, entry['pos'])
#         writer.writerow([entry['sentence'], entry['pos'], entry['span'], entry['length'], entry['aann'], entry['og_detected'], entry['current_detected'], entry['dep_parse_detected'], entry['current_with_dt_detected'], int(detected)])


with open("../data/permissed_sampled_annotated_test1_1k_final_regex.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(["sentence","pos","span","length","aann","og_detected","current_detected","dep_parse_detected","current_with_dt_detected", "final_detected"])
    for i, entry in enumerate(permissed_sampled_annotated):
        sent_tok = tokenize(entry['sentence'])
        detected, span = detect_aann_basic(sent_tok, entry['pos'])
        writer.writerow([entry['sentence'], entry['pos'], entry['span'], entry['length'], entry['aann'], entry['og_detected'], entry['current_detected'], entry['dep_parse_detected'], entry['current_with_dt_detected'], int(detected)])

# for i, entry in enumerate(permissed_sampled_annotated):
            
#         # if detected :
#             # print(span, entry['aann'])
#     sent_tok = tokenize(entry['sentence'])
#     detected, span = detect_aann_basic(sent_tok, entry['postags'])
#     if entry['aann'] == '1':
#         if detected:
#             pass
#         else:
#             print(i, entry['span'], entry['postags'])
#         # if entry['aann'] == '1':
#         #     if "few" in entry['sentence']:
#         #         print(i, entry)


In [50]:
# few_pos
print(permissed_sampled_annotated[290])

sent_toks = tokenize(permissed_sampled_annotated[290]['sentence'])
detect_aann_basic(sent_toks, permissed_sampled_annotated[290]['pos'])

{'sentence': 'Buoyed by a record 1.5 million pre-orders, the game sold 2.4 million copies in a single day, for a total of 125 million dollars.', 'pos': 'VBN IN DT NN CD CD NNS NNS NNS , DT NN VBD CD CD NNS IN DT JJ NN , IN DT NN IN CD CD NNS .', 'span': 'a record 1.5 million pre-orders, the game sold 2.4 million copies in a single day, for a total of 125 million dollars', 'length': '117', 'aann': '1', 'og_detected': '0', 'current_detected': '0', 'dep_parse_detected': '1', 'current_with_dt_detected': '0'}
VBN IN ARTICLE RECORD CD CD NNS NNS NNS , DT NN VBD CD CD NNS IN ARTICLE JJ NN , IN ARTICLE NN IN CD CD NNS .


(True, 'a record 1.5 million pre')

In [52]:
club_record = []
for i, (s, p)  in enumerate(zip(sents, postags)):
    if "club record" in s or "club-record" in s:
        club_record.append(i)

In [54]:
for i in club_record:
    st, pt = sent_tokens[i], postags[i]
    print(detect_aann_basic(st, pt))

IN DT CD NN , NNP VBD ARTICLE NN HYPH NN RECORD IN CD , VBD RP CD NNS , CC VBD ARTICLE JJ CD VBN NN NN , IN NNP VBD ARTICLE NN HYPH RECORD CD NNS CC DT RB JJ NNP NNP NNP NNP NN -LRB- RB IN DT NN , CC DT NNP NNP CC NNP NNP VBD VBN VBN IN NNP CC NNP NNPS -RRB- -RRB-
(True, 'a club - record 109 games')
DT NN VBD VBN IN NNP POS NNP NNP IN NN IN ARTICLE NN HYPH RECORD NN IN CD .
(False, '')
DT NN VBD PRP$ NNP NNP RECORD IN CD NN NNS IN CD NNS , CC VBD TO VB ARTICLE JJ JJ NN DT NN , VBG ARTICLE NN RECORD NN IN CD NNS IN ARTICLE NN WDT VBD RB DT JJ NN CC VBD IN NNP CD .
(False, '')
IN ARTICLE CD NN IN NNP NNP , DT NNPS VBD IN ARTICLE JJ NN IN DT NN IN DT NNP NNP , VBG DT NN IN ARTICLE NN IN ARTICLE NN RECORD JJ NN VBG .
(False, '')
IN JJ NN NNP NNP VBD VBN IN NNP NNP NN NNP IN ARTICLE JJ NNP NN RECORD NN IN $ CD IN NNP CD , DT NNPS VBD TO VB IN DT JJ CD NNS IN DT NN , VBG RB RB IN JJ IN ARTICLE NN IN CD NNS IN RB CD NN .
(False, '')
CD FEW NN NNS IN NNP : IN NNP , NNP NNP NNP CC NNP NNP : VBD

In [69]:
old_detected = utils.read_csv_dict("../data/babylm-aanns/aanns_indef_all.csv")
old_idx = [int(entry['sentence_idx']) for entry in old_detected]

In [70]:
new_idx = []
new_entries = []

for i, (s, st, p) in enumerate(tqdm(zip(sents, sent_tokens, postags))):
    # if i in old_idx:
    #     continue
    detected, span = detect_aann_basic(st, p)
    if detected:
        new_idx.append(i)
        new_entries.append((i, span))

11632617it [00:49, 235227.64it/s]


In [71]:
len(new_idx)

2448

In [74]:
len(set(old_idx).intersection(set(new_idx)))

1349

In [98]:
old_idx_few = []
for entry in old_detected:
    if "few" in entry['ADJ'] or "couple" in entry['ADJ'] or "several" in entry['ADJ'] or "many" in entry['ADJ'] or "more" in entry['ADJ'] or 'dozen' in entry['ADJ']:
        old_idx_few.append(int(entry['sentence_idx']))

In [99]:
len(old_idx_few)

955

In [100]:
# len((set(old_idx) - set(old_idx_few)).intersection(set(new_idx)))
(set(old_idx) - set(old_idx_few)) - set(new_idx)

{410230,
 463226,
 611182,
 1064925,
 1662035,
 1925531,
 2278535,
 2695635,
 2696537,
 4023499,
 4251011}

In [112]:
# save old union new ids
old_union_new = list(set(old_idx).union(set(new_idx)))

with open("../data/babylm-analysis/old_union_new_regex_aanns.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(["sentence_idx"])
    for idx in old_union_new:
        writer.writerow([idx])

# save new ids alone

with open("../data/babylm-analysis/new_regex_aanns.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(["sentence_idx"])
    for idx in new_idx:
        writer.writerow([idx])
